In [ ]:
import gc
import os

import numpy as np
import pandas as pd
import plotly.express as px

from tqdm.auto import tqdm
from collections import Counter
from sklearn.utils import shuffle

# from metric import PrintMetric

import warnings
warnings.filterwarnings("ignore")

raw_data_path = 'D:/git/Pigitt/vehicle_rec_sys/off_data'
new_data_path = 'D:/git/Pigitt/vehicle_rec_sys/recall_data'

os.makedirs(new_data_path, exist_ok=True)

In [ ]:
user_info = pd.read_csv(raw_data_path + '/user_info.txt', sep='\t') #, index_col=0
user_info.columns = ['user_id', 'device', 'os', 'province', 'city','age','gender']

doc_info = pd.read_table(raw_data_path + '/doc_info.txt', sep='\t')
doc_info.columns = ["item_id", "标题", "发文时间", "图片数量",  "一级分类", "二级分类", "关键词"]

all_data = pd.read_csv(raw_data_path + '/train_data_30w.txt', sep='\t')#, index_col=0
all_data.columns = ["user_id", "item_id", "展现时间", "网路环境", "刷新次数", "展现位置", "是否点击", "消费时长（秒）"]

# 预处理数据在时间上的特征

In [ ]:
all_data['展现时间'] = pd.to_datetime(all_data['展现时间'], utc=True, unit='ms').dt.tz_convert('Asia/Shanghai')

# 合并文档发文时间
all_data = all_data.merge(doc_info[['item_id', '发文时间']], how='left', on='item_id')

# 统计各文档最早展现时间
temp_df = all_data.groupby('item_id', as_index=False)['展现时间'].agg({'最早展现时间': min})
all_data = all_data.merge(temp_df, how='left', on='item_id')

In [ ]:
# 填充发文时间为空的文档
condition_row = (all_data['发文时间'].isnull()) | (all_data['发文时间'] == 'Android')
all_data.loc[condition_row, '发文时间'] = all_data.loc[condition_row, '最早展现时间']
all_data.loc[~condition_row, '发文时间'] = pd.to_datetime(
    all_data.loc[~condition_row, '发文时间'], utc=True, unit='ms').dt.tz_convert('Asia/Shanghai')

# 填充发文时间晚于最早展现时间的文档
condition_row = (all_data['发文时间'] > all_data['最早展现时间'])
all_data.loc[condition_row, '发文时间'] = all_data.loc[condition_row, '最早展现时间']

In [ ]:
# 统计日期信息
all_data['发文日期'] = pd.to_datetime(all_data['发文时间']).dt.date
all_data['展现日期'] = all_data['展现时间'].dt.day

# 划分训练集和测试集

In [ ]:
mode = 'test'

if mode == 'debug':
    all_data = shuffle(all_data)
    all_data.reset_index(drop=True)

    train_data = all_data[(all_data['展现日期'] >= 5) & (all_data['展现日期'] < 6)]
    test_data = all_data.loc[all_data['展现日期'] == 6, :]
else:
    train_data = all_data[(all_data['展现日期'] >= 1) & (all_data['展现日期'] < 6)]
    test_data = all_data.loc[all_data['展现日期'] == 6, :]

del all_data, doc_info, user_info
gc.collect()

In [ ]:
train_data.head(1)

# 对日期在2021-7-1发表的新闻进行分析

In [ ]:
# 筛选出发文日期在2021-07-01的新闻
train_71 = train_data.loc[train_data['发文日期'] == pd.to_datetime('2021-7-1'), :]
train_71.head()

In [ ]:
# 提取该天新闻的全部交互数据
train_item_71 = train_data[train_data['item_id'].isin(train_71['item_id'].unique())]

# 统计每个交互样本的发文到展示时间差
train_item_71['发文到展示时间差'] = train_item_71['展现时间'] - train_item_71['发文时间']
train_item_71['发文到展示小时差'] = train_item_71['发文到展示时间差'].dt.days * 24 + round((train_item_71['发文到展示时间差'].dt.seconds + 1) / 3600)

train_item_71.head()

In [ ]:
train_item_71.head(1)

In [ ]:
train_item_71['发文到展示小时差'].unique()

In [ ]:
# 对train_item_71按 “发文到展示小时差” 进行分组
train_item_71_g = train_item_71.groupby(['发文到展示小时差', '是否点击'], as_index=False)['是否点击'].agg(({'expos_sum': 'size'}))
train_item_71_g.head()

In [ ]:
train_item_71_g.sort_values('expos_sum', ascending=False)

In [ ]:
# 将新闻发表后不同小时内的曝光数和点击数绘制出来
fig = px.bar(train_item_71_g, x='发文到展示小时差', y='expos_sum', title='曝光数', color='是否点击')
fig.show()

+ 7月1日发文的新闻，大约在2小时后，曝光数和点击数有了明显的上升
+ 大约在32小时候，曝光和点击开始有了明显了下降
+ 在72小时后，该部分新闻基本不再曝光

In [ ]:
# 统计了各样本从发文到现在（最大展现时间）的时间间隔
max_time = train_data['展现时间'].max()

train_data['发文到现在时间差'] = max_time - train_data['发文时间']
train_data['发文到现在小时差'] = train_data['发文到现在时间差'].dt.days * 24 + (train_data['发文到现在时间差'].dt.seconds + 1) / 3600

In [ ]:
# 由于新闻具有时效性，根据时间差划分不同的训练集
train1 = train_data.loc[(train_data['发文到现在时间差'] > pd.to_timedelta('0 hours')) & (train_data['发文到现在时间差'] <= pd.to_timedelta('2 hours'))]
train2 = train_data.loc[(train_data['发文到现在时间差'] > pd.to_timedelta('2 hours')) & (train_data['发文到现在时间差'] <= pd.to_timedelta('16 hours'))]
train3 = train_data.loc[(train_data['发文到现在时间差'] > pd.to_timedelta('16 hours')) & (train_data['发文到现在时间差'] <= pd.to_timedelta('32 hours'))]
train4 = train_data.loc[(train_data['发文到现在时间差'] > pd.to_timedelta('32 hours')) & (train_data['发文到现在时间差'] <= pd.to_timedelta('72 hours'))]

# 计算新闻的热度
具体过程如下：
(1) 根据牛顿冷却公式，计算新闻随时间变化的新鲜度;
$$\boldsymbol{T}(\boldsymbol{t})=\boldsymbol{H}+\left(\boldsymbol{T}_{\mathbf{0}}-\boldsymbol{H}\right) \boldsymbol{e}^{-\boldsymbol{\alpha}\left(\boldsymbol{t}-\boldsymbol{t}_{\mathbf{0}}\right)}
$$

(2) 根据用户是否点击新闻以及新闻的总交互数，计算文档的流行度；

In [ ]:
train = pd.concat([train1, train2, train3, train4], axis=0)
rate = len(train[train['是否点击'] == 1]) / len(train[train['是否点击'] == 0])

In [ ]:
delta_t = train['发文到现在小时差']

# 新闻的初始pop_factor为10，在32小时后，pop_factor大约为1
train['pop_factor'] = 10 * np.exp(-0.072 * delta_t)

# 如果用户未点击新闻，施加惩罚
train['pop_factor'] = train[['pop_factor', '是否点击']].apply(lambda x: x['pop_factor'] if x['是否点击'] else rate * x['pop_factor'], axis=1)

# 从新闻的曝光次数上，统计新闻的热点信息
popular_items_group = train.groupby(['item_id'])['pop_factor'].sum()

# 根据新闻的流行度，构建热门新闻列表
popular_factor, popular_items = zip(*sorted(zip(popular_items_group, popular_items_group.keys()))[::-1])
popular_items = list(popular_items)

In [ ]:
# 构建用户近期点击的新闻集合，用户再次推送
positive_items_per_user1 = train1[train1['是否点击'] == 1].groupby(['user_id'])['item_id'].apply(list)
positive_items_per_user2 = train2[train2['是否点击'] == 1].groupby(['user_id'])['item_id'].apply(list)
positive_items_per_user3 = train3[train3['是否点击'] == 1].groupby(['user_id'])['item_id'].apply(list)
positive_items_per_user4 = train4[train4['是否点击'] == 1].groupby(['user_id'])['item_id'].apply(list)

In [ ]:
topk = 100

test_users = test_data['user_id'].unique()

## 策略1： 将用户观看过的新闻再次推送，不足数量的部分直接使用热门推荐（不具备个性化）

In [ ]:
outputs = {}
for user in tqdm(test_users):
    user_output = []
    # 将用户已经观看过部分的新闻，再次推送给用户
    if user in positive_items_per_user1.keys():
        most_common_items_of_user = {k: v for k, v in Counter(positive_items_per_user1[user]).most_common()}
        user_output += list(most_common_items_of_user.keys())[:min(topk // 10, len(most_common_items_of_user))]
    if user in positive_items_per_user2.keys():
        most_common_items_of_user = {k: v for k, v in Counter(positive_items_per_user2[user]).most_common()}
        user_output += list(most_common_items_of_user.keys())[:min(topk // 20, len(most_common_items_of_user))]
    if user in positive_items_per_user3.keys():
        most_common_items_of_user = {k: v for k, v in Counter(positive_items_per_user3[user]).most_common()}
        user_output += list(most_common_items_of_user.keys())[:min(topk // 30, len(most_common_items_of_user))]
    if user in positive_items_per_user4.keys():
        most_common_items_of_user = {k: v for k, v in Counter(positive_items_per_user4[user]).most_common()}
        user_output += list(most_common_items_of_user.keys())[:min(topk // 50, len(most_common_items_of_user))]

    user_output = user_output[:topk]
    # 剩余部分，使用热门新闻进行填充
    user_output += list(popular_items[:topk - len(user_output)])

    outputs[user] = user_output


In [ ]:
positive_items_test = test_data.groupby(['user_id'])['item_id'].apply(list)

test_true = []
for i, user in tqdm(enumerate(test_users)):
    test_true.append(positive_items_test[user])

test_pred = []
for user, pred in outputs.items():
    test_pred.append(pred)

PrintMetric(test_true, test_pred, topk)

## 策略2： 根据用户最近观看的新闻类别，推送相似类别的热点新闻
待续...